Keras

In [78]:
import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.models import Model
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [0]:
BATCH_SIZE=64
EPOCHS = 25
VAL_SPLIT = 0.85
CONV_DROPOUT_RATE = 0.3
FC_DROPOUT_RATE = 0.5

In [80]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
(y_train, y_test) = map(to_categorical, [y_train, y_test])
split = int(len(x_train)*VAL_SPLIT)
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]

datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True)
datagen.fit(x_train)
test_datagen = ImageDataGenerator(rescale=1./255)

170500096/170498071 [==============================] - 29s 0us/step


In [0]:
def conv_block(filters, filter_size=(3,3), pool_size=(2,2), pool_strides=(2,2), activation='relu', dropout_rate=CONV_DROPOUT_RATE):
  def _conv_block(x):
    x = Conv2D(filters, filter_size, activation=activation, padding='same')(x)    
    x = Conv2D(filters, filter_size, activation=activation, padding='same')(x)
    x = MaxPool2D(pool_size, pool_strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=dropout_rate)(x)
    return x
  return _conv_block

def dense_block(units, activation='relu', dropout_rate=FC_DROPOUT_RATE):
  def _dense_block(x):
    x = Dense(units, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=dropout_rate)(x)
    return x
  return _dense_block

In [84]:
input = Input([32,32,3])
x = conv_block(32)(input) #[32,32,3] -> [16,16,16]
x = conv_block(64)(x) #[16,16,16] -> [8,8,24]
x = conv_block(128)(x) #[8,8,24] -> [4,4,32]
x = Flatten()(x) #[4,4,128] -> [,512]
x = dense_block(512, activation='relu')(x)
out = Dense(10, activation='softmax')(x)
model = Model(input, out)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)  

In [85]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE, epochs=EPOCHS,
                    validation_data=test_datagen.flow(x_val, y_val, batch_size=BATCH_SIZE),
                    validation_steps=len(x_val) / BATCH_SIZE)
result = model.evaluate_generator(test_datagen.flow(x_test, y_test, batch_size=BATCH_SIZE), steps=len(x_test) / BATCH_SIZE)
print(result)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
665/664 [==============================] - 24s 36ms/step - loss: 2.0134 - acc: 0.3431 - val_loss: 3.5915 - val_acc: 0.2443
Epoch 2/25
665/664 [==============================] - 22s 33ms/step - loss: 1.4274 - acc: 0.4972 - val_loss: 1.5198 - val_acc: 0.4892
Epoch 3/25
665/664 [==============================] - 22s 33ms/step - loss: 1.2282 - acc: 0.5665 - val_loss: 1.3324 - val_acc: 0.5475
Epoch 4/25
665/664 [==============================] - 23s 35ms/step - loss: 1.1224 - acc: 0.6021 - val_loss: 2.0549 - val_acc: 0.4117
Epoch 5/25
665/664 [==============================] - 22s 33ms/step - loss: 1.0489 - acc: 0.6337 - val_loss: 1.6919 - val_acc: 0.5864
Epoch 6/25
665/664 [==============================] - 22s 33ms/step - loss: 0.9924 - acc: 0.6526 - val_loss: 1.1169 - val_acc: 0.6440
Epoch 7/25
665/664 [==============================] - 23s 34ms/step - loss: 0.9567 - acc: 0.6654 - val_loss: 1.4936 - val_acc: 0.5852
Epoch 8/25
665

TensorFlow

In [1]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [0]:
BATCH_SIZE=32
EPOCHS = 25
VAL_SPLIT = 0.85
CONV_DROPOUT_RATE = 0.3
FC_DROPOUT_RATE = 0.5

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
(y_train, y_test) = map(to_categorical, [y_train, y_test])
x_train = x_train / 255.
x_test = x_test / 255.
split = int(len(x_train)*VAL_SPLIT)
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]



170500096/170498071 [==============================] - 56s 0us/step


In [4]:
def flip(x):
  return tf.image.random_flip_left_right(x)
     

def color(x):
  x = tf.image.random_hue(x, 0.08)
  x = tf.image.random_saturation(x, 0.6, 1.6)
  x = tf.image.random_brightness(x, 0.05)
  x = tf.image.random_contrast(x, 0.7, 1.3)
  return x

def rotate():
  random_angles = tf.random.uniform(shape = (tf.shape(x_train)[0], ), minval = -np.pi / 4, maxval = np.pi / 4)
  return tf.contrib.image.angles_to_projective_transforms(random_angles, tf.cast(tf.shape(x_train)[1], tf.float32), tf.cast(tf.shape(x_train)[2], tf.float32))

def zoom(x):
  scales = list(np.arange(0.85, 1.0, 0.01))
  boxes = np.zeros((len(scales), 4))

  for i, scale in enumerate(scales):
    x1 = y1 = 0.5 - (0.5 * scale)
    x2 = y2 = 0.5 + (0.5 * scale)
    boxes[i] = [x1, y1, x2, y2]

  def random_crop(img):
    crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
    return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

  choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)
  return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

augmentations = [flip, color] #zoom, rotate()

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train.astype(np.int32)))
dataset = dataset.shuffle(tf.cast(tf.shape(x_train)[0], tf.int64))

for f in augmentations:
    dataset = dataset.map(lambda x, y: tf.cond(tf.random_uniform([], 0, 1) > 0.75, lambda: (f(x),y), lambda: (x,y)), num_parallel_calls=4)
dataset = dataset.map(lambda x,y: (tf.clip_by_value(x, 0, 1),y))
dataset = dataset.batch(batch_size=BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=BATCH_SIZE)

dataset_val = tf.data.Dataset.from_tensor_slices((x_val, y_val.astype(np.int32))).batch(batch_size=BATCH_SIZE)
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test.astype(np.int32))).batch(batch_size=BATCH_SIZE)




Instructions for updating:
Colocations handled automatically by placer.


In [0]:
class Train():
  
  def __init__(self, datasets):
    self.dataset, self.dataset_val, self.dataset_test = datasets
    
    self.build_model()
  
  def __conv_block(self, x, filters, filter_size=(3,3), pool_size=(2,2), pool_strides=(2,2), activation=tf.nn.relu):
    x = tf.layers.conv2d(x, filters, kernel_size=filter_size, activation=activation, padding='same')
    x = tf.layers.conv2d(x, filters, kernel_size=filter_size, activation=activation, padding='same')
    x = tf.layers.max_pooling2d(x, pool_size, pool_strides, padding='same')
    x = tf.layers.batch_normalization(x)
    x = tf.layers.dropout(x, self.conv_dropout_rate)
    return x
    
  def __fc_block(self, x, units, activation=tf.nn.relu):
    x = tf.layers.dense(x,units, activation=activation)
    x = tf.layers.batch_normalization(x)
    x = tf.layers.dropout(x, self.fc_dropout_rate)
    return x
  
  def build_model(self):
    self.iter = iter = tf.data.Iterator.from_structure(self.dataset.output_types,self.dataset.output_shapes)
    self.train_init_op = iter.make_initializer(self.dataset)    
    self.val_init_op = iter.make_initializer(self.dataset_val)    
    self.test_init_op = iter.make_initializer(self.dataset_test)

    with tf.name_scope('placeholders'):
      self.input_itr, self.labels_itr = self.iter.get_next()
      self.batch_size = tf.placeholder(tf.int64)
      self.input = tf.placeholder(tf.float32, [None, 32, 32, 3], name='input_ph')
      self.labels = tf.placeholder(tf.int32, [None, 10], name='label_ph')
      self.lr = tf.placeholder_with_default(0.001, [])    
      self.conv_dropout_rate = tf.placeholder_with_default(1., [])
      self.fc_dropout_rate = tf.placeholder_with_default(1., [])
      
    x = tf.cast(self.input_itr, tf.float32)
    with tf.name_scope('conv_blocks'):
      x = self.__conv_block(x, 32) #[32,32,3] -> [16,16,16]
      x = self.__conv_block(x, 64) #[16,16,16] -> [8,8,24]
      x = self.__conv_block(x, 128) #[8,8,24] -> [4,4,32]
      
    x = tf.layers.flatten(x) #[4,4,128] -> [,512]
    
    with tf.name_scope('fc_blocks'):
      x = self.__fc_block(x, 512)

    self.output = tf.layers.dense(x, 10, activation=tf.nn.softmax)
    self.loss = tf.losses.softmax_cross_entropy(self.labels_itr, self.output)
    self.accuracy = tf.metrics.accuracy(tf.argmax(self.labels_itr, axis=1), tf.argmax(self.output, axis=1))
    self.train = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
    
    
  def eval(self,sess, init_op, x, y):
    total_loss=[]
    sess.run(init_op, feed_dict={ self.input: x, self.labels: y, self.batch_size: BATCH_SIZE})
    for step in range(int(len(x)/BATCH_SIZE)):
      _loss, _acc,  = sess.run([self.loss, self.accuracy])
      total_loss.append(_loss)
    loss = sum(total_loss)/len(total_loss)
    return (loss, _acc)
    
      
    
  def train_model(self,epochs = 25, lr=0.01, decay=0.0):
    #iter = self.dataset.make_initializable_iterator()
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      sess.run(tf.local_variables_initializer())
      for epoch in range(epochs):
        sess.run(self.train_init_op, feed_dict={ self.input: x_train, self.labels: y_train, self.batch_size: BATCH_SIZE})
        total_train_loss=[]  
        

        for step in range(int(len(x_train)/BATCH_SIZE)):
          _, _loss, _acc_train,  = sess.run([self.train, self.loss, self.accuracy], feed_dict={self.lr:lr, self.conv_dropout_rate:0.3, self.fc_dropout_rate:0.5})
          total_train_loss.append(_loss)
        
        val_loss, val_acc = self.eval(self.val_init_op, x_val, y_val)
        #sess.run(self.val_init_op, feed_dict={ self.input: x_val, self.labels: y_val, self.batch_size: BATCH_SIZE})
        #for step in range(int(len(x_val)/BATCH_SIZE)):
        #  _loss, _acc_val,  = sess.run([self.loss, self.accuracy])
        #  total_val_loss.append(_loss)
        print('epoch:',epoch+1,'  train loss:', sum(total_train_loss)/len(total_train_loss), ' train acc:',_acc_train ,'  val loss:', val_loss, ' val acc:',val_acc)
      
      test_loss, test_acc = self.eval(self.test_init_op, x_test, y_test)
      #sess.run(self.test_init_op, feed_dict={ self.input: x_test, self.labels: y_test, self.batch_size: BATCH_SIZE})
      #total_test_loss=[]  
      #for step in range(int(len(x_test)/BATCH_SIZE)):
      #  _loss, _acc_test,  = sess.run([self.loss, self.accuracy])
      #  total_test_loss.append(_loss)
      print('test loss:', test_loss, ' test acc:',test_acc )

In [6]:
train = Train((dataset, dataset_val, dataset_test))

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [7]:
train.train_model(epochs = EPOCHS, lr=0.0002)

TypeError: ignored

PyTorch

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

In [0]:
BATCH_SIZE=32
EPOCHS = 25
VAL_SPLIT = 0.85
SHUFFLE = True
SHUFFLE_SEED = 19
CONV_DROPOUT_RATE = 0.3
FC_DROPOUT_RATE = 0.5

In [0]:
transform_train = transforms.Compose(
    [transforms.ColorJitter(brightness=0.05, contrast=(0.7,1.3), saturation=(0.6,1.6), hue=(-0.2,0.2)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

valset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_test)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(VAL_SPLIT * num_train))
#trainset, valset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
if SHUFFLE:
  np.random.seed(SHUFFLE_SEED)
  np.random.shuffle(indices)

train_idx, valid_idx = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, sampler=trainset, num_workers=4)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE,
                                          shuffle=True, sampler=valid_sampler, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 32, 3, padding=1)        
        self.conv1_2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout(CONV_DROPOUT_RATE)
        
        self.conv2_1 = nn.Conv2d(32, 64, 3, padding=1)        
        self.conv2_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout(CONV_DROPOUT_RATE)
        
        self.conv3_1 = nn.Conv2d(64, 128, 3, padding=1)        
        self.conv3_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.bn3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout(CONV_DROPOUT_RATE)
        
        self.fc4 = nn.Linear(128 * 4 * 4, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(FC_DROPOUT_RATE)
        self.output = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.dropout2(x)
        
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.dropout3(x)
 
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.dropout4(x)
        
        x = F.softmax(self.output(x), 1)
        return x


net = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0002)

In [0]:
def eval(loader):
  total_loss = 0.0
  total_acc = 0.0
  sample_len = 0
  for i, data in enumerate(valloader, 0):

    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    
    outputs = net(inputs)
    loss = criterion(outputs, labels)


    total_loss += loss.item()
    _, prediction = torch.max(outputs.data, 1)
    sample_len += len(prediction)
    total_acc += torch.sum(prediction == labels.data)
  loss = val_loss/i 
  acc = total_acc.item()/ sample_len
  return (loss, acc)

In [0]:
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    train_loss = 0.0   
    train_acc = 0.0
    train_sample_len = 0
    
    #Train steps
    net.train()
    for i, data in enumerate(trainloader, 0):

      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      _, prediction = torch.max(outputs.data, 1)
      train_sample_len += len(prediction)
      train_acc += torch.sum(prediction == labels.data)
    train_loss = train_loss/i
    train_acc = train_acc.item()/ train_sample_len
    
    #Validation steps
    net.eval()
    val_loss, val_acc = eval(valloader)
    print('epoch is :', epoch+1, '\tloss:',train_loss,  '\tacc:',train_acc)
    print('\t\tloss:',val_loss,  '\tacc:',val_acc)

#Test steps  
test_loss, test_acc = eval(testloader)
print('Test results:\tloss:',test_loss,  '\tacc:',test_acc)